In [1]:
import pandas as pd
import gc

train_df = pd.read_parquet('./data/public_train.pqt')
test_df = pd.read_parquet('./data/submit_2.pqt')

In [2]:
rm = pd.read_csv('./data_mms/rosstat-mts.csv')
peoples_total = rm.groupby('region_mts').agg({'men':'sum','women':'sum'}).reset_index()
peoples_total.columns = ['region_mts','men_count','women_count']
peoples_total['people_total'] =  peoples_total['men_count'] + peoples_total['women_count']
peoples_total['men_share'] = peoples_total['men_count'] / peoples_total['people_total']
peoples_total['women_share'] = peoples_total['women_count'] / peoples_total['people_total']
rm = rm.merge(peoples_total, on='region_mts', how='left')

rm_age = rm.groupby(['region_mts','age_bucket']).agg({'men':'sum','women':'sum',
                                                      'people_total':'first','men_share':'first',
                                                      'women_share':'first'
                                                     }).reset_index()
rm_age['people_in_bucket'] = rm_age['men'] + rm_age['women']
rm_age['men_bucket_share'] = rm_age['men'] / rm_age['people_in_bucket']
rm_age['women_bucket_share'] = rm_age['women'] / rm_age['people_in_bucket']
rm_age['people_in_bucket'] = rm_age['people_in_bucket'] / rm_age['people_total']
df_rms = []
for i in range(7):
    df = rm_age.loc[rm_age.age_bucket == i, ['region_mts','men_bucket_share','women_bucket_share','people_in_bucket']]
    df.columns = ['region_mts',f'men_bucket_share_{i}',f'women_bucket_share_{i}',f'people_in_bucket_{i}']
    df_rms.append(df)
df_rms = pd.concat(df_rms,ignore_index=True).reset_index(drop=True)
df = rm_age[['region_mts','men_share','women_share']].drop_duplicates()
df = df.merge(df_rms, on='region_mts', how='left')
df = df.groupby('region_mts').agg(max).reset_index()
df.columns = ['region_name', 'men_share', 'women_share', 'men_bucket_share_0',
       'women_bucket_share_0', 'people_in_bucket_0', 'men_bucket_share_1',
       'women_bucket_share_1', 'people_in_bucket_1', 'men_bucket_share_2',
       'women_bucket_share_2', 'people_in_bucket_2', 'men_bucket_share_3',
       'women_bucket_share_3', 'people_in_bucket_3', 'men_bucket_share_4',
       'women_bucket_share_4', 'people_in_bucket_4', 'men_bucket_share_5',
       'women_bucket_share_5', 'people_in_bucket_5', 'men_bucket_share_6',
       'women_bucket_share_6', 'people_in_bucket_6']
df = df.fillna(0)
df.fillna(0).to_csv('./data_mms/demog.csv', index=False)

In [3]:
q = pd.read_parquet('./data/competition_data_final_pqt')
def decode_from_punycode(url: str) -> str:
    try:
        return bytearray(url, "utf-8").decode("idna")
    except Exception:
        return url
q['url_host'] = q['url_host'].apply(decode_from_punycode)
train = q.loc[q.user_id.isin(train_df.user_id.unique().tolist())]

In [4]:
d_train_q = train['url_host'].value_counts().to_dict()
q['url_host_freq'] = q['url_host'].apply(lambda x: d_train_q.get(x,0))

In [5]:
import re
q['non_digits_cnt'] = q.url_host.apply(lambda x: len(re.findall(r'[^\d\.]+', x)))

In [6]:
url_flt = q.url_host_freq < 2000
q.loc[url_flt].url_host.nunique()

198113

In [7]:
def clean_url(x):
    x = '.'.join(x.split('.')[:-1])
    x = x.replace('.turbopages', '').replace('.cdn.ampproject', '')
    if x.startswith('m.') | x.startswith('m-'):
        x = x[2:]
    x = ''.join(re.findall(r'[a-zA-Zа-яА-Я]+', x))
    return x

q.loc[url_flt, 'url_host'] = q.loc[url_flt, 'url_host'].apply(clean_url)
q.url_host.nunique()

186542

In [8]:
train = q.loc[q.user_id.isin(train_df.user_id.unique().tolist())]
d_train_q = train.groupby('url_host').user_id.nunique().to_dict()
d_train_q['-1']

1824

In [9]:
q['new_url_host_freq'] = q['url_host'].apply(lambda x: d_train_q.get(x,0))
q.loc[q['url_host_freq']*1.5 < q['new_url_host_freq'] ]

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,url_host_freq,non_digits_cnt,new_url_host_freq
2613,Санкт-Петербург,Санкт-Петербург,Samsung,Galaxy S20+,hightech,smartphone,Android,74259.0,2021-10-04,evening,1,79395,193,3,684
2628,Санкт-Петербург,Санкт-Петербург,Samsung,Galaxy S20+,w,smartphone,Android,74259.0,2021-09-01,evening,1,79395,47,2,78
5702,Тюменская область,Тюмень,Xiaomi,Redmi Note 8 Pro,ru,smartphone,Android,20465.0,2021-07-08,morning,1,161323,1450,3,3482
6902,Тюменская область,Тобольск,Samsung,Galaxy A52,sprashivai,smartphone,Android,29358.0,2022-02-27,day,1,148545,131,3,323
10134,Пермский край,Пермь,Samsung,Galaxy A40 Dual,citaty,smartphone,Android,14323.0,2022-06-09,morning,2,16874,50,2,324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322897684,Белгородская область,Белгород,Apple,iPhone SE 2020,grammarian,smartphone,iOS,40612.0,2021-07-21,morning,1,260127,7,2,12
322898406,Калужская область,Обнинск,Samsung,Galaxy A30s Dual,imgfonwall,smartphone,Android,16376.0,2021-07-22,morning,1,300964,136,3,233
322898789,Калужская область,Обнинск,Samsung,Galaxy A30s Dual,imgfonwall,smartphone,Android,16376.0,2021-07-12,day,1,300964,136,3,233
322899156,Калужская область,Обнинск,Samsung,Galaxy A30s Dual,imgfonwall,smartphone,Android,16376.0,2021-07-22,morning,1,300964,149,3,233


In [10]:
q.loc[q['url_host_freq']*2 < q['new_url_host_freq'], ['url_host','url_host_freq','new_url_host_freq']].drop_duplicates()

,url_host,url_host_freq,new_url_host_freq
2613,hightech,193,684
5702,ru,1450,3482
6902,sprashivai,131,323
10134,citaty,50,324
10230,hdlordfilms,32,109
...,...,...,...
322167977,aura,0,18
322210478,пни,0,5
322342597,mvd,0,4
322537370,skyhostel,0,3


In [11]:
q.loc[q.new_url_host_freq < 3, ['url_host','url_host_freq','new_url_host_freq']].drop_duplicates().sort_values('url_host_freq')

,url_host,url_host_freq,new_url_host_freq
146798426,helenahouse,0,0
215808711,myplaycity,0,0
215799279,teucomsk,0,0
215786546,abigerusrusff,0,0
215785583,errorfix,0,0
...,...,...,...
241056360,dkdoservis,225,1
113370161,onlinewialon,242,2
9154388,historym,248,2
58281230,onlinecheboko,399,1


In [20]:
q.loc[q.url_host == 'zubki']

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,url_host_freq,non_digits_cnt,new_url_host_freq
69963906,Республика Башкортостан,Уфа,Apple,iPhone 7,zubki,smartphone,iOS,26065.0,2021-08-01,evening,1,251328,2,2,3
85332094,Москва,Москва,Xiaomi,Redmi Note 9 Pro,zubki,smartphone,Android,23629.0,2022-03-25,day,1,403184,1,2,3
249351781,Пермский край,Пермь,Apple,iPhone Xs,zubki,smartphone,iOS,86780.0,2021-07-10,morning,1,366007,2,2,3
284612999,Тюменская область,Тюмень,Apple,iPhone 11,zubki,smartphone,iOS,55674.0,2021-07-07,evening,1,407089,2,2,3


In [12]:
q.to_parquet('./data_mms/q_f2k_raw.pq')

In [13]:
q.loc[q.new_url_host_freq<20,'url_host'] = 'unknown'

del train, d_train_q, url_flt
gc.collect()

q.url_host.nunique()

23761

In [14]:
q.loc[q.url_host == 'unknown'].shape[0]

1130719

In [15]:
q['date'] = pd.to_datetime(q['date'])

In [16]:
q = q.merge(df, on='region_name', how='left')
del df
gc.collect()

45

In [17]:
import numpy as np
cat_cols = ['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name',
            'cpe_type_cd', 'cpe_model_os_type', 'part_of_day', 'url_host']

feat_dict = {}
for c in cat_cols:
    feat_dict[c] = {x: i for i,x in enumerate(q[c].unique())}
    q[c] = q[c].map(feat_dict[c])

In [18]:
q['price'] = np.log1p(q['price'].fillna(0))
q['dow'] = q['date'].dt.dayofweek
feat_dict['dow'] = {x:x for x in range(7)}
q['diff_days'] = q['date'].diff().dt.days
q['diff_days'] = q['diff_days'].fillna(0)
q['df_user'] = (q['user_id'] != q['user_id'].shift(1)).fillna(1).astype(int)
q.loc[q.df_user == 1,'diff_days'] = 0
q.drop(['df_user','date'], axis=1, inplace=True)

In [19]:
from sklearn.preprocessing import StandardScaler
num_features = ['price','diff_days'] + ['men_share', 'women_share', 'men_bucket_share_0',
       'women_bucket_share_0', 'people_in_bucket_0', 'men_bucket_share_1',
       'women_bucket_share_1', 'people_in_bucket_1', 'men_bucket_share_2',
       'women_bucket_share_2', 'people_in_bucket_2', 'men_bucket_share_3',
       'women_bucket_share_3', 'people_in_bucket_3', 'men_bucket_share_4',
       'women_bucket_share_4', 'people_in_bucket_4', 'men_bucket_share_5',
       'women_bucket_share_5', 'people_in_bucket_5', 'men_bucket_share_6',
       'women_bucket_share_6', 'people_in_bucket_6'] #
cat_features = ['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name',
                'cpe_type_cd', 'cpe_model_os_type', 'part_of_day', 'dow', 
                'url_host', 'url_host_last']
for c in num_features:
    ss = StandardScaler()
    q[c] = ss.fit_transform(q[c].values.reshape(-1,1)).flatten()

In [20]:
q = q.groupby('user_id').agg(list).reset_index()
train_df = pd.read_parquet('./data/public_train.pqt')
q = q.merge(train_df, on='user_id', how='left')
q.to_parquet('./data_mms/q_f2k_20unk.pq')

In [21]:
feat_dict['url_host']['unknown']

16